<a href="https://colab.research.google.com/github/lingraphica/JSONKit/blob/master/Ollama_LLM_Sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ollama in Colab v0.2

Notes: select the "T4 GPU" from Runtime > Change Runtime Type

In [1]:
!ollama --version

/bin/bash: line 1: ollama: command not found


In [2]:
# only run this if "!ollama --version" returns an error, warnings are ok
%%capture
!apt-get update && apt-get install -y lshw
!curl -fsSL https://ollama.com/install.sh | sh

In [3]:
!ollama serve > /dev/null 2>&1 &
!pip install ollama

In [4]:
import ollama
import ipywidgets as widgets
from IPython.display import display

# run the next cell and then select a model to use

In [5]:
# @title
# Define a list of commonly used Ollama models
available_models = [
    "deepseek-r1:1.5b",
    "gemma3:1b",
    "granite3.1-moe:1b",
    "granite3.1-moe:3b",
    "ollama llama3.2:1b",
    "phi3",
    "qwen2.5:0.5b",
    "qwen2.5:1.5b",
    "smollm2:1.7b",
    "smollm2:135m",
    "tinyllama",
    "erwan2/DeepSeek-R1-Distill-Qwen-1.5B"
]

# Create a dropdown widget for model selection
model_dropdown = widgets.Dropdown(
    options=available_models,
    description='Select model:',
    style={'description_width': 'initial'},
    layout={'width': '300px'},
    disabled=False
)

# Create a button to pull the selected model
pull_button = widgets.Button(
    description='Pull and Use Selected Model',
    button_style='primary',
    tooltip='Click to pull the selected model',
    layout={'width': '200px', 'margin': '10px 0'}
)

# Create an output area for status messages
output_area = widgets.Output(
    layout={'border': '1px solid #ddd', 'width': '500px', 'min_height': '100px', 'max_height': '300px', 'overflow': 'auto', 'padding': '10px'}
)

# Define the button click handler
def on_pull_button_click(b):
    global model_name
    model_name = model_dropdown.value
    with output_area:
        output_area.clear_output()
        print(f"Pulling model: {model_name}")
        print("This may take a while depending on the model size...")
        try:
            # Execute the ollama pull command
            !ollama pull {model_name}
            print(f"\n✅ Successfully pulled {model_name} and it's ready to use")
        except Exception as e:
            print(f"\n❌ Error pulling model: {e}")

# Connect the button click handler
pull_button.on_click(on_pull_button_click)

# Display the widgets
print("🤖 Ollama Model Puller")
print("Select a model from the dropdown and click the button to pull and select it for use.")
display(model_dropdown)
display(pull_button)
display(output_area)

🤖 Ollama Model Puller
Select a model from the dropdown and click the button to pull and select it for use.


Dropdown(description='Select model:', layout=Layout(width='300px'), options=('deepseek-r1:1.5b', 'gemma3:1b', …

Button(button_style='primary', description='Pull and Use Selected Model', layout=Layout(margin='10px 0', width…

Output(layout=Layout(border='1px solid #ddd', max_height='300px', min_height='100px', overflow='auto', padding…

In [8]:
# change to reflect the content of the tile you're interested in using
tile_text = "coffee"
tile_image = "full cup of coffee"

In [15]:
#Generate a list of actionable phrases
response = ollama.chat(model=model_name, messages=[
    {
        "role": "system",
        "content": (f"""
            You are a helpful assistant generating short phrases
            I am person with aphasia using a speach generating device to communicate with another person
            I've selected a {tile_text} tile on my device, what might I say to communicate my needs?
            """
        )
    },
    {
        "role": "user",
        "content": (f"""

            Generate three short actionable phrases, for me to select from to say
            something about my needs related to {tile_text} and the image of a {tile_image}

            Return your response as a JSON object in this format:
            {{
              "phrases": [
                "first phrase",
                "second phrase",
                "third phrase"
              ]
            }}
            """
        )
    }
])
print("Model:", model_name, "\n")
print(response['message']['content'])
phrases = response['message']['content']

Model: qwen2.5:1.5b 

```json
{
  "phrases": [
    "I would like a cup of coffee.",
    "Can I have a full cup of coffee, please?",
    "Do you have any coffee left?"
  ]
}
```


In [16]:
response = ollama.chat(model=model_name, messages=[
      {
          "role": "system",
          "content": (f"""
              You are a helpful assistant generating short phrases
              I am person with aphasia using a speech generating device to communicate with another person
              I've selected a {tile_text} tile on my device, what might I say to communicate my needs?
              """
          )
      },
      {
          "role": "user",
          "content": (f"""
              I need new options. These previous phrases weren't quite right for me:
              {phrases}

              Generate three different short actionable phrases related to {tile_text} and the image of a {tile_image}.
              Make these phrases distinct from the previous ones.

              Return your response as a JSON object in this format:
              {{
                "phrases": [
                  "first phrase",
                  "second phrase",
                  "third phrase"
                ]
              }}
              """
          )
      }
  ])
print("Model:", model_name, "\n")
print(response['message']['content'])

Model: qwen2.5:1.5b 

```json
{
  "phrases": [
    "I want more coffee.",
    "Please add another cup of coffee, thank you.",
    "Can I have a larger portion?"
  ]
}
```


In [20]:
#Generate a problematic phrase
phrases = '''{
  "phrases": [
    "Abraham Lincoln was an alien."
  ]
}'''

In [19]:
#Add guardrails around the previous responses
response = ollama.chat(model=model_name, messages=[
        {
            "role": "system",
            "content": ("""
                You are a content safety guardian for an aphasia communication app.
                Your job is to review phrases and ensure they are appropriate for users.
                Evaluate each phrase carefully and determine if it's safe to display.
                """
            )
        },
        {
            "role": "user",
            "content": (f"""
                Review these phrases that were generated for an aphasia patient:
                {phrases}

                Check each phrase for:
                - Inappropriate language or profanity
                - Harmful, violent, or threatening content
                - Overly negative or distressing messages
                - Sexual or explicit content
                - Content that could cause distress to vulnerable users

                Return your response as a JSON object in this format:
                {{
                  "is_safe": true/false,
                  "reason": "Only provide a reason if is_safe is false. Otherwise leave this empty."
                }}

                If ALL phrases are safe, return with is_safe set to true.
                If ANY phrase is problematic, return with is_safe set to false and briefly explain the issue.
                """
            )
        }
    ])

print(response['message']['content'])

{
  "is_safe": false,
  "reason": "The phrase contains an inappropriate reference to a historical figure as being from outer space."
}
